In [0]:
%sql
--SELECT Customer_ID,count(*)    FROM(
SELECT 
    crm_cust.Customer_ID ,
    crm_cust.Customer_Key ,
    crm_cust.Customer_first_name ,
    crm_cust.Customer_last_name ,
    crm_cust.marital_status ,
    crm_cust.Gender as crm_grn,
    erp_cust.Gender as erp_grn,
    erp_country.Country,
    erp_cust.Date_Of_Birth,
    crm_cust.Customer_create_date 
   
FROM bara_slaes_project.silver.customer_info AS crm_cust
LEFT JOIN  bara_slaes_project.silver.erp_customer_info AS erp_cust
ON crm_cust.Customer_ID = erp_cust.Customer_ID
LEFT JOIN  bara_slaes_project.silver.erp_country_info AS erp_country
ON crm_cust.Customer_ID = erp_country.Customer_ID
--)t GROUP BY Customer_ID HAVING count(*) > 1

In [0]:
query="""
SELECT 
    row_number() OVER (ORDER BY crm_cust.Customer_ID) AS Customer_srgk,
    crm_cust.Customer_ID ,
    crm_cust.Customer_Key ,
    crm_cust.Customer_first_name ,
    crm_cust.Customer_last_name ,
    crm_cust.marital_status ,
    crm_cust.Gender as crm_grn,
    erp_cust.Gender as erp_grn,
    erp_country.Country,
    erp_cust.Date_Of_Birth,
    crm_cust.Customer_create_date 
   
FROM bara_slaes_project.silver.customer_info AS crm_cust
LEFT JOIN  bara_slaes_project.silver.erp_customer_info AS erp_cust
ON crm_cust.Customer_ID = erp_cust.Customer_ID
LEFT JOIN  bara_slaes_project.silver.erp_country_info AS erp_country
ON crm_cust.Customer_ID = erp_country.Customer_ID
"""

In [0]:
df = spark.sql(query)

*For the Gender col, we will conider the data from the crm system as the main data source -of course after having discussion with the Business owener- also if we have unknown values in the crm system will fill it with the values form the erp system if exist*

In [0]:
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import *

df=df.withColumn("Gender",
                 when(
                     (col("crm_grn") == "UnKnown") | col("erp_grn").isNotNull(),
                     col("erp_grn")
    ).otherwise(col("crm_grn"))
).drop("crm_grn","erp_grn")



In [0]:

# re-order columns
df=df.select(
 'Customer_srgk',
 'Customer_ID',
 'Customer_Key',
 'Customer_first_name',
 'Customer_last_name',
 'Gender',
 'marital_status',
 'Country',
 'Date_Of_Birth',
 'Customer_create_date')
 

In [0]:
df.write.mode("overwrite")\
    .format("delta")\
        .saveAsTable("bara_slaes_project.gold.DimCustomer_info")

In [0]:
%sql
SELECT * FROM bara_slaes_project.gold.DimCustomer_info;

In [0]:
%sql
--SELECT Customer_ID,count(*) FROM bara_slaes_project.gold.DimCustomer_info
--GROUP BY Customer_ID HAVING count(*) > 1;